<br>
<br>

<br>

<br>


#    <center> ❄❄ Prediction of Superconducting Critical Temperature ❄❄ <center>
<br>

<img src="https://static.spektrum.de/fm/912/thumbnails/Supraleiter-schwebender-Magnet-iStock_49768936_ktsimage.jpg.3112312.jpg">

# Introduction

the goal of this notebook is modeling critical temperature of a superconductor based on its chemical properties using two regression models, Decision tree and Random forest.

The first part will tackle the theory behind the algorithms and gives a short presentation of the relative parameters and metrics.  
The second part will be reserved to the dataset preprocessing, after introducing the necessary techniques to that purpose.  
And in the last part, an evaluation of the models' performance will be presented as well as a comparative analysis to determine wich one gets to decode the pattern behind data.

<br>
<br>

> 📍📍NB: all you questions are welcome 😃😃

<br> 
<br> 
<br> 
<br> 


**What is superconductivity?**

Superconductivity is a phenomenon of exactly zero electrical resistance and expulsion of magnetic flux fields occurring in certain materials, called superconductors, when cooled below a characteristic critical temperature. Superconductors are widely used in many industries, e.g. the Magnetic Resonance Imaging (MRI) in health care, electricity transportation in energy industry and magnetic separation, etc.
Superconductivity Data Set used in this notebook is hosted in UCI machine learning [repository](https://archive.ics.uci.edu/ml/datasets/Superconductivty+Data)   

the repository contains two files:   
* (1) train.csv contains 81 features extracted from 21263 superconductors along with the critical temperature in the 82nd column, and this will be our main dataset.
* (2) unique_m.csv contains the chemical formula broken up for all the 21263 superconductors from the train.csv file. (which we will not use)


In [ ]:
#necessary import 
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sb
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, validation_curve,KFold
from sklearn.preprocessing import MinMaxScaler , Normalizer
from sklearn.metrics import  explained_variance_score, mean_squared_error,mean_absolute_error
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


# 1.Some mathematics
<br>

## Regression tree algorithm

A regression tree is built through a process known as binary recursive partitioning, which is an iterative process that splits the data into partitions or branches, and then continues splitting each partition into smaller groups as the method moves up each branch.

Given a training set $X_n=x_1, x_2, ... x_n$ with responses $Y_n=y_1, y_2, ... y_n$ 

Initially, the algorithm begins allocating the data into the first two partitions considering every splitting variable $j$ and split point $s$    
  
  
 <center> $ R_1(j,s) = \{ X| X_j \le s \} $ and $ R_2(j,s) = \{ X| X_j \gt s \} $ </center>
   
   
then the algorithm selects the binary split that minimizes   
  
   <center> $ [ \sum\limits_{x_i \in R_1(j,s)} (y_i-c_1)^2 + \sum\limits_{x_i \in R_2(j,s)} (y_i-c_2)^2 ]$ </center>
   
 with $c_k= avg(y_i | x_i\in R_k(j,s))$ ,  $k = 1,2$  
   
 <br>
   
   Having found the best split, This splitting rule is then applied to each of the resulting branches. This process continues until each node or partition reaches a user-prespecified minimum node size and becomes a terminal node. (If the quantity $\sum\limits_{x_i \in node} (y_i-c_{node})^2$ in a node is zero, then that node is considered a terminal node even if it has not reached the minimum size.)  
       

A very large tree might overfit the data, while a small tree might not capture the important structure. this tradeoff is governed by the tree size, and to find the best tree size, we use the cost complexity pruning algorithm:
> * start by growing a large tree $T_0$ and stop spplitting process only when minimum node size is reached.  
* for a given $\alpha$ find $T(\alpha) \subseteq T_0$ that minimize   
  
  
 <center> $ C_{\alpha}(T) = C(T) + \alpha|\tilde T|$ </center>    
   
   
>with $T$ define any subtree $\subset T_0$, and $|\tilde T|$  denote the number of terminal nodes in $T$,  and  $ C(T) =\sum\limits_{t \in \tilde T} N_t Q_t$ , $t$ indexes terminal nodes
  
  <center> $ Q_t= \frac 1{N_t} \sum\limits_{x_i \in R_t}(y_i-\hat c_t)^2 $ </center>    
   <center> $ N_t= cardinal\{x_i \in R_t\} $ </center>  
     
     
   <center> $ \hat c_t= \frac 1{N_t} \sum\limits_{x_i \in R_t}y_i $ </center>
     
> * try the above for different values of $\alpha$ and pick $\hat \alpha$ that minimizes the cross-validated error. It's clear that for $\alpha = 0$ the solution is the full tree $T_0$ and with increasing $\alpha$ the tree becomes smaller and smaller implying less complexity.

Once the final tree $T(\hat \alpha)$ is built, for every new observation that falls into a terminal node, the predicted response will be the mean value of responses in that node.

<br>
<br>

## Random forest regression algorithm

Random forest is a kind of regression trees bagging, the idea behind is to construct multiple trees at training time and averaging the results to end up with a single prediction model highly accurate.  
The conception of the algorithm is simple but requires a short explanation of each step:  
* For $b=1, ...,B$ :  

 * Sample from $X_n, Y_n$, with replacement, n-sized sample  $X_b, Y_b$ .  
 
 * for each bootsrapped training set $X_b, Y_b$, a random selection of $m$ predictors is chosen as split candidates from the full set of $p$ predictors, typically we choose $m= \sqrt p$.  
 
 * Train a regression tree $\hat T_b$ on $X_b, Y_b$.   
   
* for each new sample $x^* $, the predicted response is $ \hat y^* = \frac 1{B} \sum\limits_{b=1}^{B} \hat y^b$  
  
  
<br>
<br>

## Performance metrics
  
many performance metrics exist to evaluate the quality of fit of a regression model, The most commonly used are:

* MSE (Mean Squared Error), is defined as the average squared distance between the actual score and the predicted score, a good regression model is characterized by a small MSE and vice-versa

<center> $MSE= \frac { \sum\limits_{i=1}^n (y_i-\hat y_i)^2 }n $ </center>
<br>
* MAE (Mean Absolute Error), is the average of absolute differences between our target and predicted values,

<center> $MAE= \frac { \sum\limits_{i=1}^n |y_i-\hat y_i| }n $ </center>

<br>

the main difference between MSE and MAE is that this latter is less sensitive to outliers, how ?? ===>
Imagine a target with a spread of values in $[3,12] $ and some outliers ranging from $70$ to $100$, the squared error between 100 and 3 is huge $(100-3)^2=9409$ compared to the absolute error  $|100-3|=97$, which gives impression that the MSE is huge as well. The algorithm by trying to minimize this MSE it will gives more weights to outliers, this scenario will certainly lead to overfitted and biased model. 

So MAE loss is more preferable if we detect the existence of a outliers.

# 2. Data preprocessing

one of the most important steps in data preprocessing is feature selection which aims at reducing the dimensionality of the problem, and eliminate redundant or irrelevant (noisy) variables. Benefits are twofold: it decreases complexity of the predictive model, and allows more accurate prediction since high-dimensionality makes predictive models more prone to overfitting, and estimates of parameters more variant.   

Two are the main approaches to feature selection:
* Filter methods: They attempt to assess the merits of features from the data, ignoring the effects of the selected feature subset on the performance of the learning algorithm. Examples are methods that select variables by ranking them through compression techniques (like PCA), or by computing correlation or a more advanced similarity measure such as minimum redundancy maximum relevance (mRMR) with the output.   
* Wrapper methods: these methods assess subsets of variables according to their usefulness to a given predictor. The method conducts a search for a good subset using the learning algorithm itself as part of the evaluation function. The problem boils down to a problem of stochastic state space search. Example are the stepwise methods proposed in linear regression analysis.

## Correlation Analysis Between Predictors

after loading the training dataset we will try to investigate correlation between features, and if any features selection technique might be applied



In [ ]:
import requests, io, zipfile

r = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00464/superconduct.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
data = pd.read_csv(z.open('train.csv'))

In [ ]:
data.head()

In [ ]:
data.info()

Except for number_of_elements and critical temperature; all of the variables are derived using highly correlated mathematical functions; mean, weighted mean, geometric mean, weighted geometric mean, standard deviation, weighted standard deviation, entropy, weighted entropy, range and weighted range calculations. This can be shown using the correlation matrix.


In [ ]:
# display the correlation matrix
plt.figure(figsize=(15,10))
sb.heatmap(data.iloc[:,:-1].corr(),cmap='twilight_shifted')
plt.show()

Principal Component Analysis (PCA) tends to reduce the dimensions of the data. It focus  on the maximum variance amount to convert correlated variables  into a set of orthogonal - mutually uncorrelated - variables called principal components such that the first principal component accounts for the maximum proportion of the variance of the original dataset, and subsequent orthogonal components account for the maximum  proportion  of  the  remaining  variance.  The process steps of PCA are as follows:    

let $Z$ be the standardized input matrix :
 
1. Compute the mean vector of $Z$.  
2. Compute the covariance matrix of data.  
3. Compute the eigenvalue and eigenvector matrix of covariance matrix, let $E$ be the matrix of eigenvectors.
4. Sort the eigenvalues λ₁, λ₂, …, λp from largest to smallest, and then sort the eigenvectors in $E$ accordingly, let $E^*$ be the sorted matrix.
5. Form the components using $E^*$ as weighting vector. $Z^* = ZE^*$

typically we take the first components explaining at least 90% of the common variation.  

PCA technique is sensitive to the scaling of the features since it's based on the covariance matrix, a look at the summary table below shows that there are large differences between the ranges of variables in our dataset, those variables with larger ranges might dominate over those with small ranges , that is the reason why it is critical to perform standardization prior to PCA.

In [ ]:
data.describe()

### features scaling


In [ ]:
# split  dataset into test and train
x = data.iloc[:, :-1]
y = data.iloc[:, -1] 
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.1) 

In [ ]:
scalerx = MinMaxScaler()
x_train = scalerx.fit_transform(x_train)
x_test = scalerx.transform(x_test)
#scalery = MinMaxScaler()
#y_train = scalery.fit_transform(np.array(y_train).reshape(-1, 1))

### PCA application

In [ ]:
pca = PCA(n_components = 30)
pca.fit(x_train)

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)

fig, ax = plt.subplots()
xi = np.arange(1, 31, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='grey')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 31, step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.9, color='g', linestyle='-')
plt.text(0.5, 0.92, '90% of total variance', color = 'g', fontsize=14)

ax.grid(axis='x')
plt.show()

This curve quantifies how much of the total 82-dimensional variance is contained within the first 30 components. As we see, the first 10 components contain 90% of the variance, 
#### one problem here is how to choose the optimal number of components ?
that's what we will try to answer using cross validation technique in the next part

# 3.Training 

In [ ]:
# function returns MAE and MSE of the fitted model
# for each number of components between 1 and 20 using cross validation
def cross_validation(model,x_train,y_train):
    mae=[]
    mse=[]
    for n in range(1,31):
        pca = PCA(n_components = n)
        pca.fit(x_train)
        x_train_pca = pca.transform(x_train)
        scores = cross_validate(model, x_train_pca,y_train, scoring=("neg_mean_squared_error", "neg_mean_absolute_error"), cv=10)
        mse.append(-scores['test_neg_mean_squared_error'].mean())
        mae.append(-scores['test_neg_mean_absolute_error'].mean())
    d = {'number of components': list(range(1,31)),'mae':mae,'mse':mse}
    # plot mae and mse
    fig, (ax1, ax2) = plt.subplots(2, 1)
    ax1.plot(range(1,31),mae, marker='o')
    ax1.title.set_text('cv-MAE vs number of PCA components')
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax2.plot(range(1,31), mse, marker='o' )
    ax2.title.set_text('cv-MSE vs number of PCA components')
    ax2.xaxis.set_major_locator(MaxNLocator(integer=True))

    return pd.DataFrame(d)
    


In [ ]:
regressor = DecisionTreeRegressor()
cross_validation(regressor,x_train,y_train)

The best lowest cross-validation (MAE,MSE) point occurs when n=10 components which explain almost 90% of the total variance.
now we use another cross validation model to choose the best complexity parameter $\alpha$, but before let's take a look at the range of effective alpha finded by the regressor during the training time

In [ ]:
pca = PCA(n_components = 10)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
regressor = DecisionTreeRegressor(random_state=0)
path = regressor.cost_complexity_pruning_path(x_train_pca, y_train)
ccp_alphas=path.ccp_alphas
pd.DataFrame(ccp_alphas).describe()

as we see there is a huge number of values, this means that the tree is very deep and prone to overfitting. Obviously we can not gridsearch them all, but we will select some values( far enough from the extremities ) and look after the best that gives sufficient performance on the test set

In [ ]:
pca = PCA(n_components = 10)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
regressor = DecisionTreeRegressor()
alpha = np.linspace(0, 40, 20)
mae_train=[]
mae_val=[]
mse_train=[]
mse_val=[]
for a in alpha:
    regressor = DecisionTreeRegressor(ccp_alpha=a)
    scores = cross_validate(regressor, x_train_pca,y_train, scoring=("neg_mean_squared_error", "neg_mean_absolute_error"),return_train_score=True ,cv=10)
    mse_train.append(-scores['train_neg_mean_squared_error'].mean())
    mse_val.append(-scores['test_neg_mean_squared_error'].mean())
    mae_train.append(-scores['train_neg_mean_absolute_error'].mean())
    mae_val.append(-scores['test_neg_mean_absolute_error'].mean())
   
 

    
    # plot mae and mse
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(alpha,mae_train , label='train',marker='o',drawstyle="steps-post")
ax1.plot(alpha,mae_val, label='test',marker='o',drawstyle="steps-post")
ax1.title.set_text('MAE vs alpha training and testing set')
ax1.legend(loc="upper right", frameon=False)
ax2.plot(alpha, mse_train,label='train',marker='o',drawstyle="steps-post")
ax2.plot(alpha, mse_val,label='test',marker='o',drawstyle="steps-post")
ax2.title.set_text('MSE vs alpha training and testing set')
ax2.legend(frameon=False)
    

From the graph above , the best alpha value that minimizes both train and test error is 0

In [ ]:
pca = PCA(n_components = 10)
pca.fit(x_train)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)
regressor = DecisionTreeRegressor(ccp_alpha=0) 
regressor.fit(x_train_pca,y_train)
#predict the test data
y_pred = regressor.predict(x_test_pca)


In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print("mae score = ",mae)
print("mse score = ",mse)

## random forest regressor 

for the random forest regressor, there exist a considerable number of hyperparameters must be set by the user before training, this includes the number of decision trees in the forest and the number of features considered by each tree when splitting a node. to find out the probably best value candidates, we'll conduct the gridsearch technique, the results will then be used to fit the model on the whole trainnig set.

these are the parameters we're trying to adjust:
* n_estimators = number of trees in the foreset
* max_features = max number of features considered for splitting a node
* max_depth = max number of levels in each decision tree
* bootstrap = method for sampling data points (with or without replacement)

In [ ]:
model = RandomForestRegressor()

#candidate values of hyperparameters
parameters = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'n_estimators': [100, 300, 500,700]}
grid = GridSearchCV(model, parameters, cv=3, n_jobs=-1)
grid.fit(x_train_pca, y_train.ravel())
print('Best depth:',grid.best_estimator_.max_depth) 
print('Best number of features:',grid.best_estimator_.max_features)
print('Best number of trees:',grid.best_estimator_.n_estimators)

In [ ]:
#Random Forest regressor
model=RandomForestRegressor(n_estimators=500,bootstrap=True,random_state=21,verbose=0,max_depth=100,max_features=3)
model.fit(x_train_pca,y_train.ravel())
y_pred = model.predict(x_test_pca)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print("mae score = ",mae)
print("mse score = ",mse)

As expected ,the randomforest shows better performance, than simple tree, the main cause is that the decision tree is more prone to overfit, however the randomforest aggregate many decision trees to limit overfitting as well as error due to bias and therefore yield great results.